![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Serving Spark NLP with API: Fast API with LightPipelines

# Using Fast API and LightPipeline
You can serve SparkNLP + FastAPI on Docker. To do that, we will create a project with the following files:
- `Dockerfile`: Image for creating a SparkNLP + FastAPI Docker image
- `requirements.txt`: PIP Requirements 
- `entrypoint.sh`: Dockerfile entrypoint
- `content/`: folder containing FastAPI webapp and SparkNLP keys
- `content/main.py`: FastAPI webapp, entrypoint
- `content/sparknlp_keys.json`: SparkNLP keys (for Healthcare or OCR)


## Dockerfile
`Dockerfile`: Image for creating a SparkNLP + FastAPI Docker image

In [ ]:
"""
FROM ubuntu:18.04
RUN apt-get update && apt-get -y update

RUN apt-get -y update \
    && apt-get install -y wget \
    && apt-get install -y jq \
    && apt-get install -y lsb-release \
    && apt-get install -y openjdk-8-jdk-headless \
    && apt-get install -y build-essential python3-pip \
    && pip3 -q install pip --upgrade \
    && apt-get clean && rm -rf /var/lib/apt/lists/* /tmp/* /var/tmp/* \
         /usr/share/man /usr/share/doc /usr/share/doc-base

ENV PYSPARK_DRIVER_PYTHON=python3
ENV PYSPARK_PYTHON=python3

ENV LC_ALL=C.UTF-8
ENV LANG=C.UTF-8

EXPOSE 8515

COPY requirements.txt /
RUN pip install -r /requirements.txt

COPY entrypoint.sh /
RUN chmod +x /entrypoint.sh

COPY ./content/ /content/
WORKDIR content/

ENTRYPOINT ["/entrypoint.sh"]
"""

## Other files of the project

- `requirements.txt`: PIP Requirements 

In [ ]:
"""
pyspark==3.1.2
fastapi==0.70.1
uvicorn==0.16
wget==3.2
pandas
"""

- `entrypoint.sh`: Dockerfile entrypoint

In [ ]:
"""
#!/bin/bash

export_json () {
    for s in $(echo $values | jq -r 'to_entries|map("\(.key)=\(.value|tostring)")|.[]' $1 ); do
        export $s
    done
}

export_json "/content/sparknlp_keys.json"

pip install --upgrade spark-nlp-jsl==$JSL_VERSION --user --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

if [ $? != 0 ];
then
    exit 1
fi

python3 /content/main.py
"""

## Example to serve 2 pipelines
We are going to download and store in memory two pipelines: `ner_profiling_clinical` and `clinical_deidentification`. This will reduce the latency of loading the models every time.

- `content/main.py`: FastAPI webapp, entrypoint

In [ ]:
from fastapi import FastAPI
import uvicorn
import json
import os
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp
from datetime import datetime
import warnings 
warnings.filterwarnings("ignore")
from sparknlp.pretrained import PretrainedPipeline

app = FastAPI()
event_list = dict()
pipelines = {}

@app.get("/benchmark/pipeline")
async def get_one_sequential_pipeline_result(modelname, text=''):
    # print(list(pipelines.keys()))
    if modelname is None or modelname not in pipelines.keys():
        return json.dumps({'error': f"{modelname} not in loaded list: {list(pipelines.keys())}"})

    return pipelines[modelname].annotate(text)

@app.on_event("startup")
async def startup_event():
    if 'pipeline_loaded' in event_list:
        return
    event_list['start_up']=datetime.now()
    print(f'startup has been started at {datetime.now()}...', )
    print(list(pipelines.keys()))

    print(f'****** spark nlp healthcare version fired up {datetime.now()} ******')
    event_list['sparknlp_fired']=datetime.now()

    print("- App started.")
    with open('/content/sparknlp_keys.json', 'r') as f:
        license_keys = json.load(f)

    params = {'spark.driver.memory': '4G'}
    spark = sparknlp_jsl.start(secret=license_keys['SECRET'], params=params)

    # Defining license key-value pairs as local variables
    locals().update(license_keys)

    # Adding license key-value pairs to environment variables
    os.environ.update(license_keys)

    print("-- Spark NLP Version :", sparknlp.version())
    print("-- Spark NLP_JSL Version :", sparknlp_jsl.version())

    print(f'****** Loading pretrained pipelines fired up {datetime.now()} ****** ')
    pipelines['ner_profiling_clinical'] = PretrainedPipeline('ner_profiling_clinical', 'en', 'clinical/models')
    pipelines['clinical_deidentification'] = PretrainedPipeline("clinical_deidentification", "en", "clinical/models")
    event_list['pipeline_loaded'] = datetime.now()

    print(event_list)

if __name__ == "__main__":
    uvicorn.run('main:app', host='0.0.0.0', port=8515)

## Keys file

...and last, but not least, add your sparknlp_keys.json to `content/sparknlp_keys.json`!
Don't forget to fulfill with your license values.

In [ ]:
{
  "AWS_ACCESS_KEY_ID": "",
  "AWS_SECRET_ACCESS_KEY": "",
  "SECRET": "",
  "SPARK_NLP_LICENSE": "",
  "JSL_VERSION": "",
  "PUBLIC_VERSION": ""
}


## Building and running Docker
Spin up a Docker container using the SparkNLP+FastAPI Docker image we created before

In [ ]:
"""
docker build -t johnsnowlabs/sparknlp:sparknlp_api .
docker run -v jsl_keys.json:/content/sparknlp_keys.json -p 8515:8515 -it johnsnowlabs/sparknlp:sparknlp_api
"""

## Consuming the API from a Python Script

Use this code to query the API either sequentially (1 call at a time) or sending N concurrent calls using ThreadPoolExecutor

In [ ]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor

ner_text = """
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting. The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO. 
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

modelname = 'clinical_deidentification'
# modelname = 'ner_profiling_clinical'

def get_url(args):
    res = requests.get(args[0])
    return res    

In [ ]:
# 1 call
# ==================
query = f"?modelname={modelname}&text={ner_text}"
url = f"http://localhost:8515/benchmark/pipeline{query}"

print(get_url([url]))

In [ ]:
# N calls in parallel
# ==================
list_of_urls = []

N_CALLS = 10    
for i in range(0, N_CALLS):
  list_of_urls.append((url, i))

with ThreadPoolExecutor() as pool:
  response_list = list(pool.map(get_url, list_of_urls))
  print(response_list)